1. Importações e definições de variáveis de ambiente

In [44]:
# ETL RAW -> SILVER | SINISTROS PRF

from pathlib import Path
from dotenv import load_dotenv
import os
import unicodedata

import numpy as np
import pandas as pd

import psycopg2
from psycopg2.extras import execute_values


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 80)

print("ETL RAW -> SILVER | SINISTROS PRF")

# Caminhos
BASE_PATH = Path(os.getcwd()).parent.parent
RAW_PATH = BASE_PATH / "data_layer" / "raw"
SILVER_PATH = BASE_PATH / "data_layer" / "silver" / "data"
SILVER_PATH.mkdir(parents=True, exist_ok=True)

RAW_FILES = sorted([p for p in RAW_PATH.iterdir() if p.suffix.lower() == ".csv"])

print(f"Arquivos encontrados em raw: {len(RAW_FILES)}")
for p in RAW_FILES:
    print(" -", p)

# Banco (.env)
load_dotenv(BASE_PATH / ".env")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "database": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
}

print("DB host:", DB_CONFIG["host"])
print("DB port:", DB_CONFIG["port"])
print("DB name:", DB_CONFIG["database"])


ETL RAW -> SILVER | SINISTROS PRF
Arquivos encontrados em raw: 2
 - /home/matheus-brant/Desktop/referencia/SBD2-Grupo-19-PRF/data_layer/raw/dados_brutos_2024.csv
 - /home/matheus-brant/Desktop/referencia/SBD2-Grupo-19-PRF/data_layer/raw/dados_brutos_2025.csv
DB host: localhost
DB port: 5432
DB name: prf


In [45]:
# Helpers: normalização e conversões

NULL_LIKE = {"", " ", "null", "none", "nan", "na", "n/a", "(null)", "NoneType", "NaN", "NULL", "N/A"}

VALID_UF = {
    "AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS","MG",
    "PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO"
}

UNKNOWN_LIKE = {
    "ignorado",
    "nao informado",
    "nao-informado",
    "sem informacao",
    "sem-informacao",
    "desconhecido",
    "0",
}

#Limpa string: trim, espaços duplicados e nulos padrão
def normalize_text(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series([], dtype="string")

    s = s.astype("string")
    s = s.map(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)
    s = s.str.strip().str.replace(r"\s+", " ", regex=True)

    s_lower = s.str.lower()
    return s.mask(s_lower.isin(NULL_LIKE), pd.NA)

# Converte para inteiro nullable
def to_int(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s, errors="coerce").astype("Int64")

# Converte para float nullable (aceita vírgula)
def to_float(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce").astype("Float64")

# Valida faixa de latitude/longitude
def validate_coordinates(lat: pd.Series, lon: pd.Series) -> tuple[pd.Series, pd.Series]:
    lat = lat.where(lat.isna() | ((lat >= -90) & (lat <= 90)), pd.NA)
    lon = lon.where(lon.isna() | ((lon >= -180) & (lon <= 180)), pd.NA)
    return lat.astype("Float64"), lon.astype("Float64")

# Mantém UF só se for válida
def validate_uf(s: pd.Series) -> pd.Series:
    s = normalize_text(s).str.upper()
    return s.where(s.isin(VALID_UF), pd.NA).astype("string")

# Troca valores tipo 'ignorado' por NULL
def null_if_unknown(s: pd.Series) -> pd.Series:
    s = normalize_text(s)

    def _map(x):
        if x is None or pd.isna(x):
            return pd.NA
        v = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii").lower().strip()
        return pd.NA if v in UNKNOWN_LIKE else x

    return s.map(_map).astype("string")


In [46]:
# Helpers: tempo e categorias

DAY_MAP = {
    "segunda-feira": 0,
    "terca-feira": 1,
    "terça-feira": 1,
    "quarta-feira": 2,
    "quinta-feira": 3,
    "sexta-feira": 4,
    "sabado": 5,
    "sábado": 5,
    "domingo": 6,
}

# Aceita HH:MM:SS e HH:MM
def parse_time(s: pd.Series) -> pd.Series:
    s = normalize_text(s)
    t1 = pd.to_datetime(s, format="%H:%M:%S", errors="coerce")
    t2 = pd.to_datetime(s, format="%H:%M", errors="coerce")
    return t1.fillna(t2).dt.time

# Dia da semana -> número (Seg=0 ... Dom=6)
def map_weekday(s: pd.Series) -> pd.Series:
    def _norm(x):
        if x is None or pd.isna(x):
            return None
        x = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii")
        return x.strip().lower()

    s = normalize_text(s).map(_norm)
    return s.map(DAY_MAP).astype("Int64")

# Sexo -> masculino | feminino | NULL
def map_gender(s: pd.Series) -> pd.Series:
    s = normalize_text(s)

    def _map(x):
        if x is None or pd.isna(x):
            return pd.NA
        v = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii").lower().strip()
        if v in {"m", "masc", "masculino"}:
            return "masculino"
        if v in {"f", "fem", "feminino"}:
            return "feminino"
        return pd.NA

    return s.map(_map).astype("string")

#Estado físico -> ileso | leve | grave | obito | NULL
def map_physical_state(s: pd.Series) -> pd.Series:
    s = normalize_text(s)

    def _map(x):
        if x is None or pd.isna(x):
            return pd.NA
        v = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii").lower().strip()

        if "obito" in v or "morto" in v:
            return "obito"
        if "grave" in v:
            return "grave"
        if "leve" in v:
            return "leve"
        if "ileso" in v or "sem ferimentos" in v:
            return "ileso"
        return pd.NA

    return s.map(_map).astype("string")

# uso_solo: sim -> urbano | nao -> rural
def map_land_use(s: pd.Series) -> pd.Series:
    s = normalize_text(s)

    def _map(x):
        if x is None or pd.isna(x):
            return pd.NA
        v = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii").lower().strip()
        if v == "sim":
            return "urbano"
        if v == "nao":
            return "rural"
        return pd.NA

    return s.map(_map).astype("string")


In [47]:
# Helpers: features derivadas

# Idade -> faixas.
def age_bucket(age_s: pd.Series) -> pd.Series:
    age = pd.to_numeric(age_s, errors="coerce")
    age = age.mask((age <= 0) | (age > 120), np.nan)

    bins = [-0.1, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99, 10_000]
    labels = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80-89","90-99","100+"]

    return pd.cut(age, bins=bins, labels=labels).astype("string")

# Ano fabricação -> idade do veículo -> faixas
def vehicle_age_bucket(year_fab_s: pd.Series, year_ref_s: pd.Series) -> pd.Series:
    year_fab = pd.to_numeric(year_fab_s, errors="coerce")
    year_ref = pd.to_numeric(year_ref_s, errors="coerce")

    age = (year_ref - year_fab).mask(lambda x: (x < 0) | (x > 120), np.nan)

    bins = [-0.1, 4, 9, 14, 19, 29, 120]
    labels = ["0-4","5-9","10-14","15-19","20-29","30+"]

    return pd.cut(age, bins=bins, labels=labels).astype("string")

# Padroniza 'tracado_via' (remove duplicatas e melhora leitura).
def standardize_road_layout(s: pd.Series) -> pd.Series:
    s = normalize_text(s)
    stopwords = {"de", "da", "do", "das", "dos", "e"}

    def _key(txt: str) -> str:
        return unicodedata.normalize("NFKD", txt).encode("ascii", "ignore").decode("ascii").lower().strip()

    def _pretty(txt: str) -> str:
        words = [w.strip() for w in txt.split() if w.strip()]
        out = []
        for i, w in enumerate(words):
            lw = w.lower()
            out.append(lw if (i > 0 and lw in stopwords) else (w[:1].upper() + w[1:].lower()))
        return " ".join(out)

    def _map(x):
        if x is None or pd.isna(x):
            return pd.NA

        parts = [p.strip() for p in str(x).split(";") if p and p.strip()]
        if not parts:
            return pd.NA

        uniq = {}
        for p in parts:
            k = _key(p)
            if k and k not in uniq:
                uniq[k] = _pretty(p)

        ordered = [uniq[k] for k in sorted(uniq.keys())]
        return ";".join(ordered) if ordered else pd.NA

    return s.map(_map).astype("string")


In [48]:
# LOAD: funções de banco
def get_conn(db_config: dict):
    return psycopg2.connect(
        host=db_config["host"],
        port=db_config["port"],
        dbname=db_config["database"],
        user=db_config["user"],
        password=db_config["password"],
    )

def load_to_postgres(df: pd.DataFrame, db_config: dict, mode: str = "truncate"):
    cols = [
        "ano_arquivo","sinistro_id","pessoa_id","veiculo_id","data_hora","dia_semana_num",
        "uf","municipio","delegacia","latitude","longitude","causa_acidente","tipo_acidente",
        "classificacao_acidente","fase_dia","sentido_via","condicao_meteorologica","tipo_pista",
        "tracado_via","caracteristicas_via","tipo_envolvido","estado_fisico","faixa_etaria_condutor",
        "sexo_condutor","tipo_veiculo","faixa_idade_veiculo",
    ]

    df_load = df[cols].copy()
    df_load = df_load[cols].copy()

    # troca tudo que é "missing" do pandas por None
    df_load = df_load.astype(object).where(pd.notna(df_load), None)

    records = [tuple(row) for row in df_load.itertuples(index=False, name=None)]


    if not records:
        print("Nada para carregar (df vazio).")
        return

    if mode == "upsert":
        set_cols = [c for c in cols if c not in ("sinistro_id", "pessoa_id")]
        set_clause = ", ".join([f"{c}=EXCLUDED.{c}" for c in set_cols])
        insert_sql = f"""
            INSERT INTO silver.sinistros ({",".join(cols)})
            VALUES %s
            ON CONFLICT (sinistro_id, pessoa_id)
            DO UPDATE SET {set_clause}
        """
    else:
        insert_sql = f"""
            INSERT INTO silver.sinistros ({",".join(cols)})
            VALUES %s
        """

    conn = get_conn(db_config)
    cur = conn.cursor()

    try:
        cur.execute("SELECT 1;")

        if mode == "truncate":
            cur.execute("TRUNCATE TABLE silver.sinistros;")
            conn.commit()

        execute_values(cur, insert_sql, records, page_size=5000)
        conn.commit()

        print(f"Load OK: {len(records):,} linhas processadas (mode={mode})")

    finally:
        cur.close()
        conn.close()


In [49]:
# Carrega e junta os CSVs raw

def load_raw_csvs(csv_paths: list[Path]) -> pd.DataFrame:
    dfs = []
    for p in csv_paths:
        df = pd.read_csv(
            p,
            sep=";",
            encoding="ISO-8859-1",
            low_memory=False,
            dtype=str,
        )
        df["__source_file"] = p.name
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

print("\nCarregando dados Raw...")
df_raw = load_raw_csvs(RAW_FILES)

print(f"Carregado: {df_raw.shape[0]:,} linhas x {df_raw.shape[1]:,} colunas")
print("Colunas:", list(df_raw.columns))



Carregando dados Raw...


Carregado: 372,148 linhas x 36 colunas
Colunas: ['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop', '__source_file']


In [50]:
SILVER_COLUMNS = [
    "ano_arquivo",
    "sinistro_id",
    "pessoa_id",
    "veiculo_id",
    "data_hora",
    "dia_semana_num",
    "uf",
    "municipio",
    "delegacia",
    "latitude",
    "longitude",
    "causa_acidente",
    "tipo_acidente",
    "classificacao_acidente",
    "fase_dia",
    "sentido_via",
    "condicao_meteorologica",
    "tipo_pista",
    "tracado_via",
    "caracteristicas_via",
    "tipo_envolvido",
    "estado_fisico",
    "faixa_etaria_condutor",
    "sexo_condutor",
    "tipo_veiculo",
    "faixa_idade_veiculo",
]

def to_silver(df: pd.DataFrame) -> pd.DataFrame:
    print("\nINICIANDO TRANSFORM (RAW -> SILVER)")
    print(f"Shape inicial: {df.shape}")

    df = df.copy()

    # Normalização geral
    print("Normalizando texto...")
    for col in df.columns:
        df[col] = normalize_text(df[col])

    # Corrigir typo de coluna
    if "condicao_metereologica" in df.columns:
        df = df.rename(columns={"condicao_metereologica": "condicao_meteorologica"})

    df["condicao_meteorologica"] = null_if_unknown(df.get("condicao_meteorologica"))

    # IDs
    print("Convertendo IDs...")
    df["sinistro_id"] = to_int(df["id"]) if "id" in df.columns else pd.NA
    df["pessoa_id"] = to_int(df["pesid"]) if "pesid" in df.columns else pd.NA
    df["veiculo_id"] = to_int(df["id_veiculo"]) if "id_veiculo" in df.columns else pd.NA
    df["sinistro_id"] = df["sinistro_id"].where(df["sinistro_id"].isna() | (df["sinistro_id"] > 0), pd.NA)
    df["pessoa_id"]   = df["pessoa_id"].where(df["pessoa_id"].isna() | (df["pessoa_id"] > 0), pd.NA)
    df["veiculo_id"]  = df["veiculo_id"].where(df["veiculo_id"].isna() | (df["veiculo_id"] > 0), pd.NA)


    # Data/hora
    print("Convertendo data e horário...")
    df["date_dt"] = pd.to_datetime(df.get("data_inversa"), format="%Y-%m-%d", errors="coerce")
    df["time_dt"] = parse_time(df.get("horario"))

    print("Criando data_hora...")
    time_txt = df["time_dt"].astype("string").fillna("00:00:00")
    df["data_hora"] = pd.to_datetime(df["date_dt"].astype("string") + " " + time_txt, errors="coerce")

    df["ano_arquivo"] = df["data_hora"].dt.year.astype("Int64")
    df["dia_semana_num"] = map_weekday(df.get("dia_semana"))

    # Coordenadas
    print("Convertendo latitude/longitude...")
    df["latitude"] = to_float(df.get("latitude"))
    df["longitude"] = to_float(df.get("longitude"))
    df["latitude"], df["longitude"] = validate_coordinates(df["latitude"], df["longitude"])

    # Campos derivados
    df["caracteristicas_via"] = map_land_use(df.get("uso_solo"))
    df["sexo_condutor"] = map_gender(df.get("sexo"))
    df["estado_fisico"] = map_physical_state(df.get("estado_fisico"))
    df["faixa_etaria_condutor"] = age_bucket(df.get("idade"))
    df["tracado_via"] = standardize_road_layout(df.get("tracado_via"))
    df["faixa_idade_veiculo"] = vehicle_age_bucket(df.get("ano_fabricacao_veiculo"), df["ano_arquivo"])

    # UF + município
    df["uf"] = validate_uf(df.get("uf"))
    df["municipio"] = df.get("municipio").str.upper()

    # Filtro de ano
    print("Aplicando filtro ano_arquivo (2024/2025)...")
    before = len(df)
    df = df[df["ano_arquivo"].isin([2024, 2025])].copy()
    print(f"   Linhas removidas pelo filtro: {before - len(df):,}")

    # Garantir colunas finais
    print("Garantindo colunas do contrato...")
    for col in SILVER_COLUMNS:
        if col not in df.columns:
            df[col] = pd.NA
    
    before_pk = len(df)
    df = df[df["sinistro_id"].notna() & df["pessoa_id"].notna()].copy()
    print(f"   Linhas removidas por PK inválida: {before_pk - len(df):,}")


    # Deduplicação por PK
    print("Removendo duplicatas por (sinistro_id, pessoa_id)...")
    before = len(df)
    df["__completeness"] = df[SILVER_COLUMNS].notna().sum(axis=1)

    df = df.sort_values(
        ["sinistro_id", "pessoa_id", "__completeness", "data_hora"],
        ascending=[True, True, False, False],
        na_position="last",
    )

    df = df.drop_duplicates(subset=["sinistro_id", "pessoa_id"], keep="first").drop(columns="__completeness")
    print(f"   Duplicatas removidas: {before - len(df):,}")

    df_silver = df[SILVER_COLUMNS].copy()
    print(f"Shape final (silver): {df_silver.shape}")
    return df_silver

df_silver = to_silver(df_raw)
df_silver.head()



INICIANDO TRANSFORM (RAW -> SILVER)
Shape inicial: (372148, 36)
Normalizando texto...
Convertendo IDs...
Convertendo data e horário...
Criando data_hora...
Convertendo latitude/longitude...
Aplicando filtro ano_arquivo (2024/2025)...
   Linhas removidas pelo filtro: 0
Garantindo colunas do contrato...
   Linhas removidas por PK inválida: 32,967
Removendo duplicatas por (sinistro_id, pessoa_id)...
   Duplicatas removidas: 0
Shape final (silver): (339181, 26)


,ano_arquivo,sinistro_id,pessoa_id,veiculo_id,data_hora,dia_semana_num,uf,municipio,delegacia,latitude,longitude,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_meteorologica,tipo_pista,tracado_via,caracteristicas_via,tipo_envolvido,estado_fisico,faixa_etaria_condutor,sexo_condutor,tipo_veiculo,faixa_idade_veiculo
41640,2024,571772,1268971,1018215,2024-01-01 00:05:00,0,RJ,TANGUA,DEL02-RJ,-22.72936,-42.701125,Reação tardia ou ineficiente do condutor,Colisão com objeto,Com Vítimas Fatais,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,urbano,Condutor,obito,20-29,masculino,Motocicleta,0-4
41641,2024,571774,1268985,1018226,2024-01-01 00:05:00,0,GO,ANAPOLIS,DEL02-GO,-16.229185,-49.009797,Velocidade Incompatível,Colisão com objeto,Sem Vítimas,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,rural,Condutor,ileso,30-39,feminino,Automóvel,15-19
41642,2024,571777,1269020,1018251,2024-01-01 01:45:00,0,ES,SERRA,DEL02-ES,-20.172928,-40.267364,Reação tardia ou ineficiente do condutor,Colisão com objeto,Sem Vítimas,Plena Noite,Decrescente,Nublado,Múltipla,Interseção de Vias;Reta,urbano,Condutor,ileso,50-59,masculino,Caminhonete,15-19
41643,2024,571778,1269028,1018261,2024-01-01 00:45:00,0,SC,PENHA,DEL03-SC,-26.83477,-48.706151,Acumulo de água sobre o pavimento,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Chuva,Dupla,Curva,rural,Condutor,ileso,50-59,masculino,Camioneta,10-14
41644,2024,571778,1269045,1018261,2024-01-01 00:45:00,0,SC,PENHA,DEL03-SC,-26.83477,-48.706151,Acumulo de água sobre o pavimento,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Chuva,Dupla,Curva,rural,Passageiro,leve,30-39,feminino,Camioneta,10-14


In [51]:
print("\nVALIDAÇÕES")

print("Ano (value_counts):")
print(df_silver["ano_arquivo"].value_counts(dropna=False))

print("\nNulos nas chaves:")
print("sinistro_id null:", df_silver["sinistro_id"].isna().sum())
print("pessoa_id   null:", df_silver["pessoa_id"].isna().sum())

print("\n'ignorado' em condicao_meteorologica (ideal = 0):")
print((df_silver["condicao_meteorologica"].astype("string").str.lower() == "ignorado").sum())

OUTPUT_FILE = SILVER_PATH / "sinistros_silver.csv"
df_silver.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

print("\nCSV Silver salvo em:", OUTPUT_FILE)
print("Tamanho (linhas):", len(df_silver))



VALIDAÇÕES
Ano (value_counts):
ano_arquivo
2024    179114
2025    160067
Name: count, dtype: Int64

Nulos nas chaves:
sinistro_id null: 0
pessoa_id   null: 0

'ignorado' em condicao_meteorologica (ideal = 0):
0

CSV Silver salvo em: /home/matheus-brant/Desktop/referencia/SBD2-Grupo-19-PRF/data_layer/silver/data/sinistros_silver.csv
Tamanho (linhas): 339181


In [52]:
print("\nCARREGANDO NO POSTGRES (silver.sinistros)...")


# Mode:"truncate" = limpa e recarrega tudo, "upsert"   = insere/atualiza se repetir PK
load_to_postgres(df_silver, DB_CONFIG, mode="truncate")

# valida no banco
conn = get_conn(DB_CONFIG)
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM silver.sinistros;")
total = cur.fetchone()[0]
cur.close()
conn.close()

print(f"Total no banco: {total:,}")



CARREGANDO NO POSTGRES (silver.sinistros)...


Load OK: 339,181 linhas processadas (mode=truncate)
Total no banco: 339,181
